# Project 

## The Challenge
The Swedish Forest Agency has mapped high conservation value forests, though field surveys. Approximately 67 000 areas (=nyckelbiotoper in Swedish) are delimited in the database.
However, it is costly and time consuming to conduct field surveys. So it is proposed to consider national continuous cover datasets, such as the laser scanning, satellite images and aerial photos, to identify high conservation value forests.

### The Problem:
To know whether the database of forests with high conservation value (key-biotopes) could be used in machine learning to train a model to recognise similar forests. In particular, one need to answer the following:

    - Is this dataset appropriate to use as training data?
    - If yes, how should the data set be prepared to be optimal training data? For example:
        - Should the dataset be divided into subsets of sites that exhibit similar characteristics? Which characteristics?
        - Should the polygons be edited in some way to improve accuracy?
    - Is more data needed?

## Our Proposal
We choose to divide key-biotopes by the types of habitats they contain. Then run a simple learning algorithm on Laser data (stored in `.laz` files) and see whether we can distinguish areas with that include particular Key-Biotopes from others.

### Method:
Laser measurements are available for square areas, 2.5 km X 2.5 km. To facilitate analysis, we will do it in steps:

- **Step 1:** Divide these squares into smaller squares of equal areas (called henceforth tiles)
- **Step 2:** Compute the percentage of area within a tile which is occupied by key-biotopes. Then, according to these percentages, label tiles for whether they contain key-bishops or not, i.e. a positive set and a negative set.
- **Step 3:** From the laser data, compute a set of variables that characterize each tile.
- **Step 4:** Apply a classification algorithm on the resulting dataset.

### Choices:
In the following we will only attempt a simplified application of the solution. There are several parameters which will affect the subsequent learning, and -in principle- they should be fine tuned systematically or through trial and error. Instead, we will just make some naive choices.

1. We choose to analyse tiles that include coniferous forests (Barrskogar) as key-biotopes. You are welcome to pick a different habitat type.

In [1]:
HABITAT = 'Barrskogar'

2. Selecting the size of tiles.
    - If the size is small (high resolution): one gets a bigger number of tiles to compare. There is a higher probability to get tiles that are fully covered by key-biotopes and hence their characteristics will correspond perfectly to key-biotopes. However, if the size is too small a tile might fail to capture the characteristics that define the key-biotope.
    - If the size is big (low resolution): a tile with a bigger size might capture better the overall properties that define a key-biotope from everything else. Also a smaller dataset will be less expensive to analyse. However, if the size is too big we will lose all ability to differentiate key-biotopes.
    
We should try different sizes to systematically arrive to the optimum value. But, for simplicity, we will just select a to **divide each laser square into 10 X 10 smaller squares, i.e. tiles of size 250m X 250m.**

In [2]:
TILE_SIZE = 250*250 # in meters
DIV_SIDE = 10 # divide each side of a laser square by this value
DIV_AREA = 100 # divide the area into 100 squares

3. We choose to label a tile as containing a key-beat-up if 50% or more of its area is covered by key-biotopes.

In [3]:
THRESHOLD = 0.5

# Step 1:
In this step we will be concerned with laser data referenced in 'rutor_shapefiles'. Remember 'rutor_shapefiles' describe square-shaped areas in Sweden for which laser measurements are available (and stored in `.laz` files). We aim to divide each square area into 100 equal square tiles.

## 1.1 Load The Data, Explore and Filter
- To Do:
1. Import the necessary modules.
2. Load 'rutor_shapefiles' into a GeoDataFrame.
3. Explore the data, drop all uninformative columns. Note: in the following analysis, we also won't be needing any dates.
4. Read Coordinate Reference System (CRS) of the GeoDataFrame. What are the units of distance in this CRS?
5. Currently we only have access to laser data of the year 2020, i.e. folders (column 'Block') starting with the number '20' as in `20A012` to `20F050`. Filter your GeoDataFrame accordingly.

In [4]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
laser_gdf = gpd.read_file("files/rutor_shapefiles/rutor.shp")
laser_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 21659 entries, 0 to 21658
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   id          21659 non-null  object  
 1   datum       21656 non-null  object  
 2   kommentar   0 non-null      object  
 3   flyghojd    21654 non-null  object  
 4   flyghastig  21653 non-null  object  
 5   overtackni  21653 non-null  object  
 6   punkttathe  21653 non-null  object  
 7   skannerid   21653 non-null  object  
 8   skannerfab  21653 non-null  object  
 9   skannermod  21653 non-null  object  
 10  oppningsvi  21653 non-null  object  
 11  pulsfrekve  21653 non-null  object  
 12  skanningsf  21653 non-null  object  
 13  uteffekt    21653 non-null  object  
 14  square      21659 non-null  object  
 15  Block       21659 non-null  object  
 16  SkannDat    21659 non-null  object  
 17  N           21659 non-null  object  
 18  E           21659 non-null  object  
 

In [6]:
laser_gdf.nunique() # columns with few unique values are most likely non informative.

id             1680
datum           147
kommentar         0
flyghojd          1
flyghastig        3
overtackni        3
punkttathe        4
skannerid         2
skannerfab        1
skannermod        1
oppningsvi        1
pulsfrekve        1
skanningsf        6
uteffekt          1
square        21659
Block           112
SkannDat        148
N               477
E               263
Las_Namn      21659
Skanner2          1
Region            1
LOV_AVL           2
geometry      21659
dtype: int64

In [7]:
uninforming_columns = [col for col in laser_gdf.columns if (laser_gdf[col].nunique() < 100)]
uninforming_columns.extend(['id', 'datum', 'SkannDat'])
laser_gdf = laser_gdf.drop(columns = uninforming_columns) # drop the uninforming_columns

In [8]:
laser_gdf.crs # distance is measured in meters

<Projected CRS: EPSG:3006>
Name: SWEREF99 TM
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Sweden - onshore and offshore.
- bounds: (10.03, 54.96, 24.17, 69.07)
Coordinate Operation:
- name: SWEREF99 TM
- method: Transverse Mercator
Datum: SWEREF99
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [9]:
laser_gdf = laser_gdf[(laser_gdf.Block >= '20A012') & (laser_gdf.Block <= '20F050')]

## 1.2 Split
- To Do:
1. Write a function that produces a rectangular polygon from four values (x_min, y_min, x_max, y_max). (x_min, y_min) is lower left corner, and (x_max, y_max) is the upper right corner.
        - Import the needed module from `shapely`.
2. Similar to 1, write a function that returns a list of $nXn$ rectangles instead of one. $n$ is some integer.
3. Write a function that produces a list of $nXn$ squares from a square polygon.
        - `polygon.bounds` gives the values (x_min, y_min, x_max, y_max)
4. Now utilize the function from the last point. Write a new empty GeoDataFrame. Fill its 'geometry' by dividing the polygons from the geometry of GeoDataFrame from the previous section. Add CRS information to the new GeoDataFrame.
5. Fill the data of the new GeoDataFrame with the corresponding values from columns: 'square', 'Block', 'Las_Namn'. Add a new code to identify each tile.

In [10]:
from shapely.geometry import Polygon

In [11]:
def rectangle(x_min, y_min, x_max, y_max):
    return Polygon([[x_min, y_min], [x_max, y_min], [x_max, y_max], [x_min, y_max]])

In [12]:
def n_rectangles(x_min, y_min, x_max, y_max, n):
    x_pts = np.linspace(x_min, x_max, n+1)
    y_pts = np.linspace(y_min, y_max, n+1)
    geo = []
    for i in range(n):
        for j in range(n):
            geo.append(rectangle(x_pts[i], y_pts[j], x_pts[i+1], y_pts[j+1]))
          
    return geo

def xy_indices(n):
    x_ind = []
    y_ind = []
    
    for i in range(n):
        for j in range(n):
            x_ind.append(i)
            y_ind.append(j)
            
    return [x_ind, y_ind]

In [13]:
def poly_to_squares(poly, n):
    x_min, y_min, x_max, y_max = poly.bounds
    return n_rectangles(x_min, y_min, x_max, y_max, n)

In [14]:
%%time

tiles_gdf = gpd.GeoDataFrame(data = {'square': [sq 
                                                for sq in laser_gdf.square for tile in range(DIV_AREA)],
                                    'Block': [blk
                                             for blk in laser_gdf.Block for tile in range(DIV_AREA)],
                                    'Las_Namn': [nm
                                                for nm in laser_gdf.Las_Namn for tile in range(DIV_AREA)],
                                    'x': [x
                                         for i in range(len(laser_gdf)) for x in xy_indices(DIV_SIDE)[0]],
                                    'y': [y
                                         for i in range(len(laser_gdf)) for y in xy_indices(DIV_SIDE)[1]],
                                    'Tile_ID': [sq + '_x%.2i_y%.2i'%(ind[0], ind[1])
                                               for sq in laser_gdf.square for ind in np.transpose(xy_indices(DIV_SIDE))]},
                            geometry=[tile
                                     for poly in laser_gdf.geometry for tile in poly_to_squares(poly, DIV_SIDE)],
                            crs= 'epsg:3006')

Wall time: 22.2 s


# Step 2:
Now that we have divided the area into tiles, it's time to determine how much of their area is occupied by coniferous forests key-biotopes. Remember 'keybiotopes_habitatgroups_shapefiles':
- These shapefiles are the result of the first set of exercises analysing sksNyckelBiotoper_shapefiles. Both shapefiles display the locations and attributes of the key biotopes (Nyckelbiotoper) in Sweden.

In this step we will find the subset of key-biotopes that contain coniferous forests (a subset of 'keybiotopes_habitatgroups_shapefiles'), then find the intersection between this subset and the tiles from step 1.

## 2.1 Load and Filter Data
- To Do:
1. Read 'keybiotopes_habitatgroups_shapefiles' into a GeoDataFrame.
2. Find the subset of keybiotopes which include coniferous forests (Barrskogar).

In [15]:
keybiotopes_gdf = gpd.read_file("files/keybiotopes_habitatgroups_shapefiles/keybiotopes_habitatgroups.shp")
keybiotopes_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 66915 entries, 0 to 66914
Data columns (total 24 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   Beteckn    66915 non-null  object  
 1   LanKod     66915 non-null  object  
 2   Lan        66915 non-null  object  
 3   KommunKod  66915 non-null  object  
 4   Kommun     66915 non-null  object  
 5   Objnamn    56651 non-null  object  
 6   Hektar     66915 non-null  float64 
 7   Datinv     66915 non-null  object  
 8   Biotop1    66907 non-null  object  
 9   Biotop2    16493 non-null  object  
 10  Biotop3    351 non-null    object  
 11  Beskrivn1  60235 non-null  object  
 12  Beskrivn2  47317 non-null  object  
 13  Beskrivn3  32946 non-null  object  
 14  Beskrivn4  20487 non-null  object  
 15  Beskrivn5  12015 non-null  object  
 16  Beskrivn6  6697 non-null   object  
 17  Beskrivn7  3664 non-null   object  
 18  Beskrivn8  1764 non-null   object  
 19  Url        66915 

In [16]:
subset_gdf = keybiotopes_gdf[(keybiotopes_gdf.BioGrp1 == 'Barrskogar')|(keybiotopes_gdf.BioGrp2 == 'Barrskogar')|
                                  (keybiotopes_gdf.BioGrp3 == 'Barrskogar')]

In [17]:
subset_gdf = subset_gdf.loc[:, ['Beteckn', 'geometry']]

## 2.2 Intersection and Key-biotope Ratio
- To Do:
1. Find the intersection between the subset of key-biotopes and laser tiles.
2. The area of each tile is TILE_SIZE, find the ratio occupied by key-biotopes for each tile
        -  use function `groupby`
3. Get a column with these ratios and add it to the GeoDataFrame of all tiles. Note tiles not in the intersection will have ratio = 0

In [18]:
print(tiles_gdf.crs, subset_gdf.crs)

epsg:3006 epsg:3006


In [19]:
intersection_gdf = gpd.overlay(tiles_gdf, subset_gdf, how= 'intersection')

In [20]:
from shapely.ops import unary_union

In [21]:
t = intersection_gdf.loc[:, ['Tile_ID', 'geometry']].groupby('Tile_ID').agg(lambda x: unary_union(x))

In [22]:
t['bio_ratio'] = t.geometry.apply(lambda x: x.area/TILE_SIZE)
t

,geometry,bio_ratio
Tile_ID,,
62775_3750_25_x09_y02,"MULTIPOLYGON (((377464.328 6278240.160, 377460...",0.047291
62775_3750_25_x09_y03,"POLYGON ((377289.666 6278250.000, 377286.174 6...",0.454847
62775_3775_25_x00_y03,"POLYGON ((377500.000 6278288.559, 377502.844 6...",0.001010
62825_3700_25_x03_y07,"MULTIPOLYGON (((370942.366 6284302.254, 370937...",0.156014
62825_3700_25_x03_y08,"MULTIPOLYGON (((370956.004 6284502.310, 370964...",0.369917
...,...,...
71475_8050_25_x08_y04,"POLYGON ((807121.408 7148750.000, 807118.660 7...",0.183541
71475_8050_25_x08_y05,"POLYGON ((807015.225 7148750.000, 807013.429 7...",0.351542
71475_8075_25_x03_y04,"POLYGON ((808500.000 7148517.546, 808498.342 7...",0.326622


In [23]:
t[(t.bio_ratio > 1)| (t.bio_ratio <= 0)] # check that bio_ratio doesn't have strange values

,geometry,bio_ratio
Tile_ID,,


In [24]:
tiles_gdf['bio_ratio'] = [0]*len(tiles_gdf)

tiles_gdf.set_index('Tile_ID', inplace= True)

tiles_gdf['bio_ratio'] = tiles_gdf['bio_ratio'].add(t['bio_ratio'], fill_value = 0)

tiles_gdf.reset_index(inplace= True)

In [25]:
tiles_gdf

,Tile_ID,square,Block,Las_Namn,x,y,geometry,bio_ratio
0,65100_3175_25_x00_y00,65100_3175_25,20B022,20B022_65100_3175_25.laz.laz,0,0,"POLYGON ((317500.000 6510000.000, 317750.000 6...",0.0
1,65100_3175_25_x00_y01,65100_3175_25,20B022,20B022_65100_3175_25.laz.laz,0,1,"POLYGON ((317500.000 6510250.000, 317750.000 6...",0.0
2,65100_3175_25_x00_y02,65100_3175_25,20B022,20B022_65100_3175_25.laz.laz,0,2,"POLYGON ((317500.000 6510500.000, 317750.000 6...",0.0
3,65100_3175_25_x00_y03,65100_3175_25,20B022,20B022_65100_3175_25.laz.laz,0,3,"POLYGON ((317500.000 6510750.000, 317750.000 6...",0.0
4,65100_3175_25_x00_y04,65100_3175_25,20B022,20B022_65100_3175_25.laz.laz,0,4,"POLYGON ((317500.000 6511000.000, 317750.000 6...",0.0
...,...,...,...,...,...,...,...,...
569595,70500_7275_25_x09_y05,70500_7275_25,20F035,20F035_70500_7275_25.laz.laz,9,5,"POLYGON ((729750.000 7051250.000, 730000.000 7...",0.0
569596,70500_7275_25_x09_y06,70500_7275_25,20F035,20F035_70500_7275_25.laz.laz,9,6,"POLYGON ((729750.000 7051500.000, 730000.000 7...",0.0
569597,70500_7275_25_x09_y07,70500_7275_25,20F035,20F035_70500_7275_25.laz.laz,9,7,"POLYGON ((729750.000 7051750.000, 730000.000 7...",0.0
569598,70500_7275_25_x09_y08,70500_7275_25,20F035,20F035_70500_7275_25.laz.laz,9,8,"POLYGON ((729750.000 7052000.000, 730000.000 7...",0.0


## 2.3 Which Tiles Contain Key-biotopes?
- To Do:
1. Add a new column to the tiles' GeoDataFrame that labels a tile as 'contains key-biotopes' (value = 1) or 'doesn't contain key-biotopes (value = 0). This should depend on whether area occupied by key-biotopes in a tile exceed the threshold 50% of the total area.

In [26]:
tiles_gdf['is_KeyBio'] = tiles_gdf['bio_ratio'].apply(lambda x: 1 if x >= THRESHOLD else 0)

## 2.4 Write The Result
- To Do:
1. Write the tiles' GeoDataFrame into a shapefile to use in the next steps of the project.

In [27]:
tiles_gdf.to_file('files/tiles_shapefiles/tiles.shp')